<a href="https://colab.research.google.com/github/icculp/FPG/blob/main/FPG_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install web3

In [2]:
!pip install jsonschema==3.2.0

In [3]:
import web3
import requests
import os

In [4]:
os.environ['WEB3_INFURA_PROJECT_ID'] = '3869a1c0f68f40d19ea46789566b97f9'

In [5]:
infura_url='https://ropsten.infura.io/v3/3869a1c0f68f40d19ea46789566b97f9' 
w3 = web3.Web3(web3.Web3.HTTPProvider(infura_url))

In [6]:
def approve(token, spender_address, wallet_address, private_key, max_amount=None):
    '''
        used this as my starting point, then stumbled through docs and stackoverflow posts to figure out the rest:
        https://stackoverflow.com/questions/65994914/how-to-approve-a-token-for-spending-on-uniswap-router-contract

        this will create the approval transaction, but still need to connect to node and send tx to the network
        I created a new account in metamask to provide the private key
    '''

    spender = spender_address #web3.toWei 
    if max_amount is None:
        max_amount = web3.toWei(2**64-1, 'ether') # assuming should be in eth
    #else:
    #    max_amount = web3.toWei(max_amount, 'ether')
    nonce = w3.eth.getTransactionCount(wallet_address)

    tx = token.functions.approve(spender, max_amount).buildTransaction({
        'from': wallet_address, 
        'nonce': nonce,
        })
    gas = w3.eth.estimateGas(tx)
    print(gas)
    tx_with_gas = token.functions.approve(spender, max_amount).buildTransaction({
        'from': wallet_address, 
        'nonce': nonce,
        'gas': gas
        })
    signed_tx = w3.eth.account.signTransaction(tx_with_gas, private_key)
    tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction) 
    # print(tx_hash)
    return w3.toHex(tx_hash)

In [7]:
# '0x07865c6E87B9F70255377e024ace6630C1Eaa37F'
# '0xfe724a829fdf12f7012365db98730eee33742ea2'
# '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48' mainnet contract, only an address on ropsten, ran into weird issues
USDC_Contract_Hash = web3.Web3.toChecksumAddress('0xFE724a829fdF12F7012365dB98730EEe33742ea2')
print(USDC_Contract_Hash)
Metamask_test_wallet_public_key = '0x882DC36ea09Fa3028A9D7aa63873059960b761ec'
Metamask_test_wallet_private_key = 'ab0aa0d8ed12c75a241d4188593859c8bfdf07ad07dafcaeb2a8c0b0ea6037e9'
Uni_V3_Router = '0xE592427A0AEce92De3Edee1F18E0157C05861564'
ABI = requests.get('https://gist.githubusercontent.com/veox/8800debbf56e24718f9f483e1e40c35c/raw/f853187315486225002ba56e5283c1dba0556e6f/erc20.abi.json').json()

# ABI = '[{"inputs":[{"internalType":"address","name":"account","type":"address"},{"internalType":"address","name":"minter_","type":"address"},...'

0xFE724a829fdF12F7012365dB98730EEe33742ea2


In [8]:
USDC_Contract_Instance = w3.eth.contract(USDC_Contract_Hash, abi=ABI)

approval_tx_hash = approve(
    token=USDC_Contract_Instance,
    spender_address=Uni_V3_Router,
    wallet_address=Metamask_test_wallet_public_key,
    private_key=Metamask_test_wallet_private_key,
    max_amount=100
)

26126


In [9]:
print("TX Hash for approval transaction: \n{}".format(approval_tx_hash))

TX Hash for approval transaction: 
0x265e71d42250de32463016de59c331e3294a455197995a92e8e3ea4fcf992313


In [10]:
# Verify allowance
USDC_Contract_Instance.functions.allowance(Metamask_test_wallet_public_key, Uni_V3_Router).call()

100